In [1]:
import os
import json
from dataclasses import dataclass
from typing import Dict, List

import random
import numpy as np
import torch
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    default_data_collator,
)

from peft import LoraConfig, get_peft_model


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
@dataclass
class TrainConfig:
    # 1) 모델 & 데이터
    model_name_or_path: str = "meta-llama/Meta-Llama-3-8B-Instruct"  # 원하는 베이스 LLM 이름
    train_jsonl: str = "prepared_data/train.jsonl"          # 학습시킬 json prompt
    
    # 결과 저장 위치: EBS 볼륨 내 현재 작업 디렉토리 기준 "./model"
    output_dir: str = "./model"

    # 2) 토크나이즈 & 길이
    max_length: int = 200

    # 3) 학습 하이퍼파라미터
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 16
    num_train_epochs: int = 3
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03

    # 4) 로깅 & 저장
    logging_steps: int = 50
    save_steps: int = 500
    seed: int = 42
    # 5) 기타
    dataloader_num_workers: int = 2  # Colab이면 0~2 정도로 유지

    # device_map="auto"
    # load_in_4bit=True

cfg = TrainConfig()
cfg


TrainConfig(model_name_or_path='meta-llama/Meta-Llama-3-8B-Instruct', train_jsonl='prepared_data/train.jsonl', output_dir='./model', max_length=200, per_device_train_batch_size=1, gradient_accumulation_steps=16, num_train_epochs=3, learning_rate=0.0002, warmup_ratio=0.03, logging_steps=50, save_steps=500, seed=42, dataloader_num_workers=2)

In [3]:
def resolve_jsonl_path(path: str) -> str:
    """
    - s3://bucket/key 형식이면 /tmp/train.jsonl 로 다운로드 후 해당 경로 반환
    - 그 외에는 로컬 경로 그대로 반환
    """
    if path.startswith("s3://"):
        no_scheme = path[5:]
        bucket, key = no_scheme.split("/", 1)

        local_path = "/tmp/train.jsonl"
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        s3 = boto3.client("s3")
        s3.download_file(bucket, key, local_path)
        print(f"[INFO] downloaded {path} -> {local_path}")
        return local_path
    else:
        return path


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


seed_everything(cfg.seed)


In [4]:
class JsonlSftDataset(Dataset):
    """
    SFT용 JSONL Dataset.
    각 row는 {"prompt": "...", "completion": "..."} 형태여야 함.
    반환되는 input_ids, attention_mask, labels는 모두 동일 길이를 보장해야 한다.
    """

    def __init__(self, jsonl_path, tokenizer, max_length=1024):
        self.samples = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                if not line.strip():
                    continue
                obj = json.loads(line)
                self.samples.append(obj)

        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        prompt = " ".join(str(item["prompt"]).split())
        completion = " ".join(str(item["completion"]).split())

        # 1) 프롬프트 + completion 합치기
        full_text = prompt + completion

        # 2) tokenizer로 일괄 encoding (truncation=True)
        enc = self.tokenizer(
            full_text,
            truncation=True,
            max_length=self.max_length,
            add_special_tokens=True,
        )

        input_ids = enc["input_ids"]
        attn_mask = enc["attention_mask"]

        # 3) prompt만 tokenize해서 길이 파악
        prompt_ids = self.tokenizer(
            prompt,
            add_special_tokens=False,
        )["input_ids"]

        # BOS 토큰 보정
        bos_extra = 1 if (len(input_ids) > 0 and input_ids[0] == self.tokenizer.bos_token_id) else 0

        prompt_len = len(prompt_ids) + bos_extra

        # 4) labels = input_ids 복사
        labels = input_ids.copy()

        # prompt 구간 -100
        for i in range(prompt_len):
            if i < len(labels):
                labels[i] = -100

        # ※ 여기서 중요한 부분: 
        # input_ids, attention_mask, labels 모두 정확히 동일 길이
        # collator가 나중에 batch padding 할 것이므로 Dataset에서는 길이를 맞추기만 하면 된다.

        assert len(input_ids) == len(labels), f"label mismatch: {len(input_ids)} vs {len(labels)}"

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attn_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
        }


In [5]:
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download

api = HfApi()

try:
    hf_hub_download(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        filename="config.json"
    )
    print("Access OK!")
except Exception as e:
    print("Access error:", e)

import requests

HF_TOKEN = "hf_xxxxxxxxx"   # 너의 것 넣기
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

url = "https://huggingface.co/api/models/meta-llama/Meta-Llama-3-8B-Instruct"
res = requests.get(url, headers=headers)

print(res.status_code)
# print(res.json())


Access OK!
200


In [5]:
# 1) 토크나이저
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=True,
)

# pad 토큰 세팅 (없는 모델들 대비)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [6]:
# 2) 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda",  # GPU 있으면 GPU, 없으면 CPU
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # 학습 시에는 False 권장

# 3) LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # LLaMA 계열 기준. 다른 모델이면 target_modules 이름만 바꿔주면 됨.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [8]:
import torch
from torch.nn.utils.rnn import pad_sequence

# pad token이 없는 모델이면 eos를 pad로 재사용
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

def sft_collate_fn(batch):
    """
    batch: list of {"input_ids": 1D tensor, "attention_mask": 1D tensor, "labels": 1D tensor}
    를 받아서, padding된 배치를 반환.
    """

    input_ids_list = [item["input_ids"] for item in batch]
    attention_mask_list = [item["attention_mask"] for item in batch]
    labels_list = [item["labels"] for item in batch]

    # 배치 안에서 가장 긴 길이에 맞춰 패딩
    input_ids_padded = pad_sequence(
        input_ids_list,
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
    )

    attention_mask_padded = pad_sequence(
        attention_mask_list,
        batch_first=True,
        padding_value=0,          # 패딩 위치는 0
    )

    labels_padded = pad_sequence(
        labels_list,
        batch_first=True,
        padding_value=-100,       # loss에서 무시될 값
    )

    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask_padded,
        "labels": labels_padded,
    }


In [9]:
# S3 또는 로컬 경로를 실제 로컬 파일로 resolve
train_jsonl_local = resolve_jsonl_path(cfg.train_jsonl)
print("Train JSONL local path:", train_jsonl_local)

# Dataset 생성
from torch.utils.data import Subset
from transformers import DataCollatorWithPadding

# 1) 전체 dataset 생성
full_dataset = JsonlSftDataset(
    jsonl_path=train_jsonl_local,
    tokenizer=tokenizer,
    max_length=cfg.max_length,
)

# 2) train / val split (예: 90% / 10%)
n = len(full_dataset)
val_ratio = 0.1
val_size = max(1, int(n * val_ratio))

indices = list(range(n))
import random
random.shuffle(indices)

val_indices = indices[:val_size]
train_indices = indices[val_size:]

train_dataset = Subset(full_dataset, train_indices)
eval_dataset = Subset(full_dataset, val_indices)

print(f"Total samples: {n}, train: {len(train_dataset)}, val: {len(eval_dataset)}")

# 3) collator (우리가 만든 labels를 그대로 유지)
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,           # 배치 내 최장 길이에 맞춰 패딩
    # max_length=cfg.max_length,  # (선택) 강제로 이 길이까지 패딩/컷팅하고 싶으면 설정
)


Train JSONL local path: prepared_data/train.jsonl
Total samples: 226, train: 204, val: 22


In [10]:
import transformers
print(transformers.__version__)


4.57.3


In [12]:
# trainer
from transformers import TrainingArguments

os.makedirs(cfg.output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    overwrite_output_dir=True,

    num_train_epochs=cfg.num_train_epochs,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    learning_rate=cfg.learning_rate,
    warmup_ratio=cfg.warmup_ratio,

    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    save_total_limit=3,

    fp16=False,
    bf16=True,

    seed=cfg.seed,
    report_to="none",
    dataloader_num_workers=cfg.dataloader_num_workers,

    # 추가: epoch마다 eval_loss 계산
    eval_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,   # ← 추가
    data_collator=data_collator,
)

print("Trainer ready.")



Trainer ready.


In [ ]:
for i in range(0, len(train_dataset)):
    
    sample = train_dataset[i]
    print(len(sample["input_ids"]), len(sample["labels"]))

In [14]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,No log,0.699912
2,No log,0.151138
3,No log,0.119790


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=39, training_loss=0.8509030464367989, metrics={'train_runtime': 139.1183, 'train_samples_per_second': 4.399, 'train_steps_per_second': 0.28, 'total_flos': 5516622161510400.0, 'train_loss': 0.8509030464367989, 'epoch': 3.0})

In [15]:
import os
from datetime import datetime

def save_metrics_with_timestamp(df, prefix, out_dir="./logs"):
    # 1) 디렉토리 생성
    os.makedirs(out_dir, exist_ok=True)

    # 2) 타임스탬프 생성
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 3) 파일 경로 생성
    filename = f"{prefix}_{ts}.csv"
    path = os.path.join(out_dir, filename)

    # 4) CSV 저장
    df.to_csv(path, index=False, encoding="utf-8-sig")

    print(f"Saved metrics to: {path}")
    return path

In [16]:
#epcoh별 eval_loss와 perplexity 정리
import math
import pandas as pd

log_history = trainer.state.log_history

rows = []
for log in log_history:
    # eval 단계 로그만 골라서 사용
    if "eval_loss" in log:
        epoch = log.get("epoch", None)
        eval_loss = log["eval_loss"]
        perplexity = math.exp(eval_loss)
        rows.append({
            "epoch": epoch,
            "eval_loss": eval_loss,
            "perplexity": perplexity,
        })

df_metrics = pd.DataFrame(rows)
filename = "lora_metrics"
save_metrics_with_timestamp(df_metrics, filename)
print(df_metrics)


Saved metrics to: ./logs/lora_metrics_20251206_031304.csv
   epoch  eval_loss  perplexity
0    1.0   0.699912    2.013576
1    2.0   0.151138    1.163157
2    3.0   0.119790    1.127260


In [17]:
trainer.save_model(cfg.output_dir)      # LoRA 어댑터 포함한 모델 저장
tokenizer.save_pretrained(cfg.output_dir)

print("LoRA fine-tuning 완료. 저장 위치:", cfg.output_dir)


LoRA fine-tuning 완료. 저장 위치: ./model


In [6]:
#simple test
from peft import PeftModel

# 베이스 모델 다시 로드 (새 세션이거나, 검증용)
base_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.float16,
    device_map="cuda",
)
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.use_cache = True  # inference에서는 True로 켜도 됩니다.

lora_model = PeftModel.from_pretrained(
    base_model,
    cfg.output_dir,
)
lora_model.eval()

def generate_comment(prompt_text: str, max_new_tokens: int = 256):
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=cfg.max_length,
    ).to(lora_model.device)

    with torch.no_grad():
        outputs = lora_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


test_prompt = """당신은 미국 주식 애널리스트다.

[종목 코드] NVDA
[기간] 2025-12-05

[OHLCV 요약]
- 시가: 370.5398
- 고가: 372.5257
- 저가: 367.9274
- 종가: 367.9755
- 거래량: 7391068.0

[VQ 코드 시퀀스]
[0.021593764424324,	0.1893500536680221,	0.0058695869520306,	0.0947726070880889,	-9.261945399163935e-28,	0.0349617674946785,	0.4011827707290649,	0.5699447393417358]

[뉴스 헤드라인]
- "나스닥 종합지수는 화요일 1.5% 이상 폭락한 후 수요일 1.2% 하락했습니다. S&P 500과 다우존스 산업평균지수는 모두 0.8% 하락했습니다."

위 정보를 종합하여 300~400자 분량의 시황 코멘트를 작성하라.

### 답변:
"""
comment = generate_comment(test_prompt)
print(comment)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

당신은 미국 주식 애널리스트다.

[종목 코드] NVDA
[기간] 2025-12-05

[OHLCV 요약]
- 시가: 370.5398
- 고가: 372.5257
- 저가: 367.9274
- 종가: 367.9755
- 거래량: 7391068.0

[VQ 코드 시퀀스]
[0.021593764424324,	0.1893500536680221,	0.0058695869520306,	0.0947726070880889,	-9.261945399163935e-28,	0.0349617674946785,	0.4011827707290649,	0.5699447393417358]

[뉴스 헤드라인]
- "나스닥 종합지수는 화요일 1.3% 상승, 기술株 주도"
- "오피스 소프트웨어 기업, 다양한 제품 출시 계획 발표"
- "신기술 기업, 다양한 인수합병 및 투자 유치 발표"

핵심 주제: 2025년 초 다양한 산업의 혁신 및 성장 전망. 기업들은 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다.

핵심 주제 주제: 기업의 혁신 및 성장 전망, 시장 동향 및 기업 실적. 다양한 산업의 기업들이 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다. 기업들의 혁신 및 성장 전망은 시장 동향 및 기업 실적과 직접적으로 연관이 있습니다.

핵심 주제 주제: 기업의 혁신 및 성장 전망, 시장 동향 및 기업 실적. 다양한 산업의 기업들이 다양한 제품 및 서비스 출시, 인수합병 및 투자 유치 등을 통해 기업 가치 및 성장 전망을 높이고 있습니다. 기업들의 혁
